# Comparative Analysis

## Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [2]:
df_original = pd.read_csv("../data/raw/data_regiao_hidro.csv")

In [3]:
df = df_original.assign(
    lat = df_original.posicao.apply(lambda x: eval(x)[0]),
    lon = df_original.posicao.apply(lambda x: eval(x)[1]),
    ano = df_original.data.apply(lambda x: int(x[:4])),
    mes = df_original.data.apply(lambda x: int(x[5:7]))
).drop(columns=["posicao", "data", "regiao_hidro"], axis=1)

In [4]:
df

,pr,divergencia,umidade,vento_vertical,vorticidade,fluxo_energia,EMI,nino3,atn,ats,atlgrad,atl3,seta,nesta,lat,lon,ano,mes
0,68.92,-2.849130e-06,74.14,-0.01,0.000007,93.54,0.49,-0.63,0.28,-0.22,0.50,-0.32,-0.21,0.14,-4.75,-39.25,1981,1
1,59.98,-3.877240e-06,73.94,-0.02,0.000008,120.53,0.49,-0.63,0.28,-0.22,0.50,-0.32,-0.21,0.14,-4.75,-39.00,1981,1
2,54.32,-4.451220e-06,73.52,-0.03,0.000011,75.55,0.49,-0.63,0.28,-0.22,0.50,-0.32,-0.21,0.14,-4.75,-38.75,1981,1
3,34.91,-4.331980e-06,73.19,-0.02,0.000014,24.54,0.49,-0.63,0.28,-0.22,0.50,-0.32,-0.21,0.14,-4.75,-38.50,1981,1
4,60.35,-2.912150e-06,74.28,0.07,0.000006,81.54,0.49,-0.63,0.28,-0.22,0.50,-0.32,-0.21,0.14,-4.50,-39.00,1981,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96325,12.81,7.554250e-07,78.69,-0.04,0.000015,97.69,0.04,-1.17,0.45,0.40,0.05,0.12,0.35,0.51,-5.25,-40.25,2022,2
96326,27.71,1.383160e-06,79.00,-0.13,0.000006,-135.32,0.04,-1.17,0.45,0.40,0.05,0.12,0.35,0.51,-5.00,-40.75,2022,2
96327,24.11,-1.849470e-07,79.46,-0.09,-0.000002,120.67,0.04,-1.17,0.45,0.40,0.05,0.12,0.35,0.51,-5.00,-40.50,2022,2
96328,16.98,2.199350e-06,78.71,-0.05,0.000014,292.69,0.04,-1.17,0.45,0.40,0.05,0.12,0.35,0.51,-5.00,-40.25,2022,2


In [5]:
X = df.drop(columns=["pr"], axis=1)
y = df[["pr"]]

In [6]:
X

,divergencia,umidade,vento_vertical,vorticidade,fluxo_energia,EMI,nino3,atn,ats,atlgrad,atl3,seta,nesta,lat,lon,ano,mes
0,-2.849130e-06,74.14,-0.01,0.000007,93.54,0.49,-0.63,0.28,-0.22,0.50,-0.32,-0.21,0.14,-4.75,-39.25,1981,1
1,-3.877240e-06,73.94,-0.02,0.000008,120.53,0.49,-0.63,0.28,-0.22,0.50,-0.32,-0.21,0.14,-4.75,-39.00,1981,1
2,-4.451220e-06,73.52,-0.03,0.000011,75.55,0.49,-0.63,0.28,-0.22,0.50,-0.32,-0.21,0.14,-4.75,-38.75,1981,1
3,-4.331980e-06,73.19,-0.02,0.000014,24.54,0.49,-0.63,0.28,-0.22,0.50,-0.32,-0.21,0.14,-4.75,-38.50,1981,1
4,-2.912150e-06,74.28,0.07,0.000006,81.54,0.49,-0.63,0.28,-0.22,0.50,-0.32,-0.21,0.14,-4.50,-39.00,1981,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96325,7.554250e-07,78.69,-0.04,0.000015,97.69,0.04,-1.17,0.45,0.40,0.05,0.12,0.35,0.51,-5.25,-40.25,2022,2
96326,1.383160e-06,79.00,-0.13,0.000006,-135.32,0.04,-1.17,0.45,0.40,0.05,0.12,0.35,0.51,-5.00,-40.75,2022,2
96327,-1.849470e-07,79.46,-0.09,-0.000002,120.67,0.04,-1.17,0.45,0.40,0.05,0.12,0.35,0.51,-5.00,-40.50,2022,2
96328,2.199350e-06,78.71,-0.05,0.000014,292.69,0.04,-1.17,0.45,0.40,0.05,0.12,0.35,0.51,-5.00,-40.25,2022,2


In [7]:
X.shape

(96330, 17)

## Treinando os Modelos

In [8]:
n_splits = 30
models = {
    'Linear': LinearRegression(),
    'KNN': KNeighborsRegressor(n_neighbors=3),
    'Árvore de Decisão': DecisionTreeRegressor(),
    'Xgboost': XGBRegressor()
}


preprocessing = ColumnTransformer([
    ("ohe", OneHotEncoder(), ["mes"]),
    ("scaler", StandardScaler(), X.columns.to_list()[:-1])
])


scores = {}
for model_name, model in models.items():
    model_score = cross_validate(
        estimator = Pipeline(steps=[
            ("preprocessing", preprocessing),
            ("model", model)
        ]),
        X = X,
        y = y,
        cv = ShuffleSplit(n_splits=n_splits, test_size=.2),
        scoring = [
            'neg_mean_absolute_error',
            'neg_mean_squared_error',
            'neg_root_mean_squared_error',
            'r2'
        ]
    )
    if not(scores):
        scores = {key: [] for key in model_score}
        scores['model'] = []
    for key in model_score:
        scores[key].extend(model_score[key])
    scores['model'].extend([model_name] * n_splits)

## Avaliando os modelos

In [9]:
pd.DataFrame(scores).groupby('model').agg([np.mean, np.std]).transpose()

model                                         KNN       Linear     Xgboost  \
fit_time                         mean    0.033738     0.075384    1.783977   
                                 std     0.002916     0.006989    0.069433   
score_time                       mean   24.239427     0.014775    0.017454   
                                 std     0.158583     0.003548    0.001863   
test_neg_mean_absolute_error     mean  -12.724187   -32.393602  -13.399571   
                                 std     0.138137     0.199130    0.137582   
test_neg_mean_squared_error      mean -595.389053 -2266.162058 -503.788964   
                                 std    15.266231    38.510810   11.717889   
test_neg_root_mean_squared_error mean  -24.398650   -47.602570  -22.443777   
                                 std     0.313382     0.403560    0.261008   
test_r2                          mean    0.923895     0.712091    0.935530   
                                 std     0.001767     0.002777    0.001641   

model                                  Árvore de Decisão  
fit_time                         mean           0.923048  
                                 std            0.018855  
score_time                       mean           0.013178  
                                 std            0.000719  
test_neg_mean_absolute_error     mean         -15.658495  
                                 std            0.176495  
test_neg_mean_squared_error      mean        -925.625977  
                                 std           26.190426  
test_neg_root_mean_squared_error mean         -30.421145  
                                 std            0.431448  
test_r2                          mean           0.881501  
                                 std            0.003543